<a href="https://colab.research.google.com/github/Tamiz07/Demand-Forecasting-Project/blob/main/Demand_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### Data Cleaning :

In [ ]:
df = pd.read_csv("/content/train 2.csv")
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.info()

In [ ]:
monthly_sales = df.groupby(pd.Grouper(key='date', freq='M')).sum().reset_index()

In [ ]:
monthly_sales.head(10)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(monthly_sales['date'], monthly_sales['sales'])
plt.xlabel('Date')
plt.ylabel('sales')
plt.title('Monthly Customer Sales')
plt.show()

In [ ]:
monthly_sales['sales_diff'] = monthly_sales['sales'].diff()
monthly_sales.head(10)

In [ ]:
monthly_sales = monthly_sales.dropna()
monthly_sales.head(10)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(monthly_sales['date'], monthly_sales['sales_diff'])
plt.xlabel('Date')
plt.ylabel('Sales_diff')
plt.show()

In [ ]:
data = monthly_sales.drop(['date', 'item', 'store', 'sales'], axis=1)

In [ ]:
for i in range(1, 13):
  col_name = 'month_' + str(i)
  data[col_name] = data['sales_diff'].shift(i)
data = data.dropna().reset_index(drop=True)
data.head(10)

Split the data into train and test

In [ ]:
data.info()

In [ ]:
train_data = data[:-3]
test_data = data[-3:]
print('Train Data Shape : ', train_data.shape)
print('Test Data Shape : ', test_data.shape)

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [ ]:
x_train, y_train = train_data[:,1:], train_data[:,0:1]
x_test, y_test = test_data[:,1:], test_data[:,0:1]
y_train = y_train.ravel()
y_test = y_test.ravel()
print('X_train shape : ', x_train.shape)
print('Y_train shape : ', y_train.shape)
print('X_test shape : ', x_test.shape)
print('Y_test shape : ', y_test.shape)

Make prediction dataframe to merge the predicted sales price of all trained algs

In [ ]:
sales_dates = monthly_sales['date'][-3:].reset_index(drop=True)
predict_df = pd.DataFrame(sales_dates)

In [ ]:
act_sales = monthly_sales['sales'][-3:].to_list()
print(act_sales)

To create linear regression model and predicted output

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)
lr_pre = lr_model.predict(x_test)

In [ ]:
lr_pre = lr_pre.reshape(-1, 1)
lr_pre_test = np.concatenate([lr_pre, x_test], axis=1)
lr_pre_test = scaler.inverse_transform(lr_pre_test)

In [ ]:
result = []
for i in range(0, len(lr_pre_test)):
  result.append(lr_pre_test[i][0] + act_sales[i])
lr_pre_set = pd.Series(result, name='Linear Prediction')
predict_df = predict_df.merge(lr_pre_set, left_index=True, right_index=True)

In [ ]:
predict_df

In [ ]:
lr_mse = np.sqrt(mean_squared_error(predict_df['Linear Prediction'], monthly_sales['sales'][-3:]))
lr_mae = mean_absolute_error(predict_df['Linear Prediction'], monthly_sales['sales'][-3:])
lr_r2 = r2_score(predict_df['Linear Prediction'], monthly_sales['sales'][-3:])
print('Linear Regression MSE : ', lr_mse)
print('Linear Regression MAE : ', lr_mae)
print('Linear Regression R2 : ', lr_r2)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(monthly_sales['date'], monthly_sales['sales'])
plt.plot(predict_df['date'], predict_df['Linear Prediction'])
plt.title('Customer sales Forecast using LR Model')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend(['Actual sales', 'Predicted sales'])
plt.show()